In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import datetime
from tqdm import trange
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
import time
from numpy import *
from sklearn.linear_model import  *
from matplotlib import pyplot as plt 
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import KFold, RepeatedKFold
from xgboost import *
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import *
from sklearn.linear_model import  *
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sympy import *
warnings.filterwarnings('ignore')

# 工具函数

In [6]:
dict={}
dict['四川省']=32617
dict['安徽省']=32001
dict['福建省']=58145
dict['河北省']=38909
dict['贵州省']=23151
dict['山东省']=56885
dict['黑龙江省']=37697
dict['广东省']=58833
dict['北京']=94648
dict['重庆']=43223
dict['浙江省']=68805
dict['辽宁省']=61996
dict['河南省']=34211
dict['海南省']=35663
dict['湖南省']=36943
dict['甘肃省']=24539
dict['陕西省']=43117
dict['内蒙古自治区']=67836
dict['江苏省']=75354
dict['天津']=100105
dict['上海']=90993
dict['云南省']=25322
dict['湖北省']=42826
dict['江西省']=31930
dict['吉林省']=47428
dict['山西省']=34984
dict['宁夏回族自治区']=39613
dict['西藏自治区']=26326
dict['新疆维吾尔自治区']=37553
dict['广西壮族自治区']=30741
dict['青海省']=36875
ave=0
total=0
for key in dict:
    total+=dict[key]
ave = total/len(dict)

In [8]:
def extract_month(s):
    return s.apply(lambda x: int(x.split('-')[1]))

In [18]:
# 合并数据
temp1 = pd.read_csv('../Data/round1_diac2019_train.csv',low_memory=False)
temp2 = pd.read_csv('../Data/round2_diac2019_train.csv',low_memory=False)
temp = temp1.append(temp2)
temp.to_csv('../Data/round12_diac2019_train.csv',index=False)

In [19]:
# 读取原始数据
trian = pd.read_csv('../Data/round12_diac2019_train.csv',low_memory=False) # 读取初赛加复赛数据
trian1 = pd.read_csv('../Data/round2_diac2019_train.csv',low_memory=False) #读取复赛数据
all_customer = pd.DataFrame(trian[['customer_id']]).drop_duplicates(['customer_id']).dropna()
print(all_customer.shape)

trian['order_pay_time'] = pd.to_datetime(trian['order_pay_time'])
trian['order_pay_date'] = trian['order_pay_time'].dt.date
trian1['order_pay_time'] = pd.to_datetime(trian1['order_pay_time'])
trian1['order_pay_date'] = trian1['order_pay_time'].dt.date
#validata_date_begin = trian['order_pay_date'].max() - datetime.timedelta(days=180)

trian['order_month'] = extract_month(trian['order_pay_date'].astype(str))
trian1['order_month'] = extract_month(trian['order_pay_date'].astype(str))

trian["GDP"]=ave
trian1["GDP"]=ave
for key in dict:
    trian["GDP"][trian["customer_province"]==key]=dict[key]
    trian1["GDP"][trian["customer_province"]==key]=dict[key]

count_fea=['order_month','goods_id']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    trian[col_name] = trian[feat].map(trian[feat].value_counts().astype(int))
    trian.loc[trian[col_name] < 2, feat] = -1
    trian[feat] += 1
    trian[col_name] = trian[feat].map(trian[feat].value_counts().astype(int))
    trian[col_name] = (trian[col_name] - trian[col_name].min()) / (trian[col_name].max() - trian[col_name].min())
    
    trian1[col_name] = trian1[feat].map(trian1[feat].value_counts().astype(int))
    trian1.loc[trian1[col_name] < 2, feat] = -1
    trian1[feat] += 1
    trian1[col_name] = trian1[feat].map(trian1[feat].value_counts().astype(int))
    trian1[col_name] = (trian1[col_name] - trian1[col_name].min()) / (trian1[col_name].max() - trian1[col_name].min())

train_history = trian[(trian['order_pay_date'].astype(str)<='2013-10-31')]
online_history = trian[(trian['order_pay_date'].astype(str)<='2013-12-31')]
train_label = trian[trian['order_pay_date'].astype(str)>='2013-11-01']
online_history1 = trian1[(trian1['order_pay_date'].astype(str)<='2013-12-31')]

(1826575, 1)


# 相关数据处理

In [20]:
# 删除类别唯一的特征
for df in [train_history,online_history,online_history1]:
    df.drop(['order_detail_id','order_id','goods_id','member_id'], axis=1, inplace=True)

In [21]:
# 删除某一类别占比超过90%的列
for df in [train_history,online_history,online_history1]:
    good_cols = list(df.columns)
    for col in df.columns:
        rate = df[col].value_counts(normalize=True, dropna=False).values[0]
        if rate > 0.9:
            good_cols.remove(col)
            print(col,rate)

is_customer_rate 0.9493003981690669
order_detail_goods_num 0.9538791201830489
order_detail_status 0.9102991657674617
order_status 0.9126025565943487
is_customer_rate 0.9408900062182921
order_detail_goods_num 0.9584204020672824
order_detail_status 0.9049027110138655
order_status 0.9070163430226283
order_status 0.903986967045243
is_customer_rate 0.9697505120089369
order_detail_status 0.9013975051200893
order_detail_goods_num 0.9406911189722584


In [22]:
#删除异常值
train_history = train_history[train_history['order_amount']<=2000]
train_history = train_history[train_history['order_total_payment']<=2000]

In [23]:
#添加一些特征 
train_history['goods_num'] = train_history['order_total_num']*train_history['order_count']
train_label['goods_num'] = train_label['order_total_num']*train_label['order_count']
online_history['goods_num'] = online_history['order_total_num']*online_history['order_count']
online_history1['goods_num'] = online_history1['order_total_num']*online_history1['order_count']

In [24]:
for df in [train_history,train_label,online_history,online_history1]:
    df['log_order_amount'] = np.log(df['order_amount'].values+1)
    df['log_goods_price'] = np.log(df['goods_price'].values+1)
    df['pay_deleta_time'] = ( pd.to_datetime(df['goods_list_time'])  - pd.to_datetime(df['order_pay_time']))
    df['goods_list_time'] = ( pd.to_datetime(df['goods_delist_time'])-pd.to_datetime(df['goods_list_time']))
    df['pay_deleta_time'] = df['pay_deleta_time'].dt.days+1
    df['goods_list_time'] = df['goods_list_time'].dt.days+1

# 特征工程

In [25]:
# 简单的特征生成部分代码
def make_feature_and_label(date1,date2,isSubmit):
    date1['count'] = 1
    # 统计这个用户出现了多少次
    customer_id = date1.groupby(['customer_id'],as_index=False)['count'].agg({'count':'count'})
    # 统计这个用户购买商品的价格信息
    good_price = date1.groupby(['customer_id'],as_index=False)['goods_price'].agg({'goods_price_max':'max',
                                                                                    'goods_price_min':'min',
                                                                                    'goods_price_mean':'mean',
                                                                                    'goods_price_sum':'sum'})
    
    #添加用户性别
    customer_gender= date1.groupby(['customer_id'],as_index=False)['customer_gender'].mean()
    customer_gender = customer_gender.fillna(0)
    
    
    class_le = LabelEncoder()    
    #添加用户所属省份
    date1['customer_province'].fillna('未知',inplace=True)
    date1['customer_province_l'] = class_le.fit_transform(date1['customer_province'])
    customer_province_l = date1.groupby(['customer_id'],as_index=False)['customer_province_l'].max()
    
    #gdp
    gdp = date1.groupby(['customer_id'],as_index=False)['GDP'].max()
    #月份特征
    order_month_count = date1.groupby(['customer_id'],as_index=False)['order_month_count'].max()
    
    #商品特征
    goods_id_count = date1.groupby(['customer_id'],as_index=False)['goods_id_count'].max()
    
    #统计该用户是不是会员
    is_member = date1.groupby(['customer_id'],as_index=False)['is_member_actived'].median()
    is_member = is_member.fillna(0)
    
    #统计用户购物车内商品数量
    goods = date1.groupby(['customer_id'],as_index=False)['order_count'].max()
    goods=goods.fillna(0)
    
    #统计用户花了多少钱及多少个
    order_total_num = date1.groupby(['customer_id'],as_index=False)['order_total_num'].mean()
    order_total_num = order_total_num.fillna(0)
    
    order_count = date1.groupby(['customer_id'],as_index=False)['order_count'].mean()
    order_count = order_count.fillna(0)
    
    total_num = order_count*order_total_num
    
    order_amount = date1.groupby(['customer_id'],as_index=False)['order_amount'].max()
    order_amount = order_amount.fillna(0)
    
    order_total_payment = date1.groupby(['customer_id'],as_index=False)['order_total_payment'].mean()
    order_total_payment = order_total_payment.fillna(0)
    order_total_payment_level = order_total_payment
    order_total_payment_level['order_total_payment'] = order_total_payment['order_total_payment']>293
      
    
    goods_num = date1.groupby(['customer_id'],as_index=False)['goods_num'].mean()
    goods_num.fillna(0)
    
    goods_list_time = date1.groupby(['customer_id'],as_index=False)['goods_list_time'].agg({'goods_list_time_last':'max',
                                                                                           'goods_list_time_mean':'mean'})
    
        
    pay_deleta_time = date1.groupby(['customer_id'],as_index=False)['pay_deleta_time'].agg({'pay_deleta_time_last':'max',
                                                                                            'pay_deleta_time_first':'min',
                                                                                           'pay_deleta_time_mean':'mean'})
    
    # 统计这个用户的订单最后一次购买时间
    last_time = date1.groupby(['customer_id'],as_index=False)['order_pay_date'].agg({'order_pay_date_last':'max','order_pay_date_first':'min'})
    # 当然这里面还可以构造更多的特征 
    '''
                    在这里疯狂加特征！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
    '''
    data = pd.merge(customer_id,good_price,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,is_member,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,customer_gender,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,order_month_count,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,goods_id_count,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,gdp,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,pay_deleta_time,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,goods_list_time,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,goods_num,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,order_total_num,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,order_total_payment_level,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,customer_province_l,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,last_time,on=['customer_id'],how='left',copy=False)
    data['long_time'] = pd.to_datetime(data['order_pay_date_last']) - pd.to_datetime(data['order_pay_date_first'])
    data['long_time'] = data['long_time'].dt.days + 1
    del data['order_pay_date_first']
    if isSubmit==False:
        data['order_pay_date_last'] = pd.to_datetime(date2['order_pay_date'].min()) - pd.to_datetime(data['order_pay_date_last'])
        data['order_pay_date_last'] = data['order_pay_date_last'].dt.days + 1
        data['label'] = 0
        data.loc[data['customer_id'].isin(list(date2['customer_id'].unique())),'label'] = 1
        print(data['label'].mean())
    else:
        data['order_pay_date_last'] = pd.to_datetime('2013-12-31') - pd.to_datetime(data['order_pay_date_last'])
        data['order_pay_date_last'] = data['order_pay_date_last'].dt.days + 1
    print(data.shape)
    return data

In [26]:
train = make_feature_and_label(train_history,train_label,False)
submit = make_feature_and_label(online_history,None,True)
submit1 = make_feature_and_label(online_history1,None,True)

0.20205092623801202
(1824639, 23)
(1826575, 22)
(1826575, 22)


In [27]:
train.to_csv('../Feature/train.csv',index=False)
submit.to_csv('../Feature/submit12.csv',index=False)
submit1.to_csv('../Feature/submit2.csv',index=False)